# Data collected

In [1]:
# Collecting Price data
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def get_available_data(stock):
    """Collects available data from a yfinance Ticker object."""
    other_data = {}
    other_data["income_statement_yearly"] = stock.income_stmt if stock.income_stmt is not None and not stock.income_stmt.empty else None
    other_data["income_statement_quarterly"] = stock.quarterly_income_stmt if stock.quarterly_income_stmt is not None and not stock.quarterly_income_stmt.empty else None
    other_data["balance_sheet_yearly"] = stock.balance_sheet if stock.balance_sheet is not None and not stock.balance_sheet.empty else None
    other_data["balance_sheet_quarterly"] = stock.quarterly_balance_sheet if stock.quarterly_balance_sheet is not None and not stock.quarterly_balance_sheet.empty else None
    other_data["cashflow_yearly"] = stock.cashflow if stock.cashflow is not None and not stock.cashflow.empty else None
    other_data["cashflow_quarterly"] = stock.quarterly_cashflow if stock.quarterly_cashflow is not None and not stock.quarterly_cashflow.empty else None
    other_data["analyst_price_targets"] = stock.analyst_price_targets if stock.analyst_price_targets is not None and not stock.analyst_price_targets.empty else None
    other_data["earnings_estimates"] = stock.earnings_estimate if stock.earnings_estimate is not None and not stock.earnings_estimate.empty else None
    other_data["revenue_estimates"] = stock.revenue_estimate if stock.revenue_estimate is not None and not stock.revenue_estimate.empty else None
    other_data["earnings_history"] = stock.earnings_history if stock.earnings_history is not None and not stock.earnings_history.empty else None
    other_data["eps_trend"] = stock.eps_trend if stock.eps_trend is not None and not stock.eps_trend.empty else None
    other_data["eps_revisions"] = stock.eps_revisions if stock.eps_revisions is not None and not stock.eps_revisions.empty else None
    other_data["growth_estimates"] = stock.growth_estimates if stock.growth_estimates is not None and not stock.growth_estimates.empty else None
    other_data["recommendations"] = stock.recommendations if stock.recommendations is not None and not stock.recommendations.empty else None
    other_data["info"] = stock.info if stock.info else None #fixed
    other_data["sustainability"] = stock.sustainability if stock.sustainability is not None and not stock.sustainability.empty else None
    other_data["news"] = stock.news if stock.news else None #fixed
    other_data["major_holders"] = stock.major_holders if stock.major_holders is not None and not stock.major_holders.empty else None
    other_data["institutional_holders"] = stock.institutional_holders if stock.institutional_holders is not None and not stock.institutional_holders.empty else None
    other_data["mutualfund_holders"] = stock.mutualfund_holders if stock.mutualfund_holders is not None and not stock.mutualfund_holders.empty else None
    other_data["insider_purchases"] = stock.insider_purchases if stock.insider_purchases is not None and not stock.insider_purchases.empty else None
    other_data["insider_transactions"] = stock.insider_transactions if stock.insider_transactions is not None and not stock.insider_transactions.empty else None
    other_data["insider_roster_holders"] = stock.insider_roster_holders if stock.insider_roster_holders is not None and not stock.insider_roster_holders.empty else None
    other_data["earnings_dates"] = stock.earnings_dates if stock.earnings_dates is not None and not stock.earnings_dates.empty else None
    other_data["calendar"] = stock.calendar if stock.calendar is not None and len(stock.calendar) > 0 else None
    other_data["sec_filings"] = stock.sec_filings if stock.sec_filings is not None and len(stock.sec_filings) > 0 else None
    other_data["options"] = stock.options if stock.options else None
    other_data["shares_full"] = stock.get_shares_full() if stock.get_shares_full() is not None and not stock.get_shares_full().empty else None
    other_data["funds_data"] = stock.get_funds_data() if stock.get_funds_data() is not None else None
    return other_data

def get_stock_data(stock_ids, start_date, end_date):
    """
    Fetches stock data, calculates returns, and attempts to get industry, size, and beta.

    Note: Getting reliable industry, size, and beta programmatically without paid APIs is challenging.
    This function provides best-effort approximations.
    """
    all_data = []
    data_isy = {}
    data_bsy = {}
    data_bsq = {}
    data_cy = {}
    data_ee = {}
    data_re = {}
    data_et = {}
    data_er = {}
    data_ge = {}
    data_r = {}
    data_i = {}
    data_s = {}
    data_n = {}
    data_mh = {}
    data_ih = {}
    data_ip = {}
    data_ed = {}
    data_c = {}
    data_o = {}
    data_sf = {}
    data_fd = {}

    for stock_id in stock_ids:
        try:
            stock_data = yf.download(stock_id, start=start_date, end=end_date)
            if stock_data.empty:
                print(f"No data found for {stock_id}")
                continue

            new_stock_data = pd.DataFrame({'stock_id': stock_id}, index=stock_data.index)
            new_stock_data['Date'] = stock_data.index

            # Reset the index to make the multi-index columns into regular columns
            stock_data = stock_data.reset_index()

            # Concatenate the levels of the multi-index columns
            stock_data.columns = ['Date'] + [f'{col[1]}_{col[0]}' for col in stock_data.columns[1:]]


            new_stock_data['Open'] = stock_data[f"{stock_id}_Open"].values
            new_stock_data['Close'] = stock_data[f"{stock_id}_Close"].values
            new_stock_data['High'] = stock_data[f"{stock_id}_High"].values
            new_stock_data['Low'] = stock_data[f"{stock_id}_Low"].values
            new_stock_data['Volume'] = stock_data[f"{stock_id}_Volume"].values
            new_stock_data['returns'] = new_stock_data['Close'].pct_change()

            # 2. Dividends and Splits (add to backtest_df) [Final decided not to extract: all NaN for HK stock market data]
            #dividends = stock.dividends
            #splits = stock.splits

            #if not dividends.empty:
            #    new_stock_data["Dividends"] = dividends

            #if not splits.empty:
            #    new_stock_data["Splits"] = splits

            # Attempt to get industry (very basic - relies on yfinance info)
            ticker = yf.Ticker(stock_id)
            info = ticker.info
            industry = info.get('industry', 'Unknown')
            new_stock_data['industry'] = industry

            # Attempt to get market cap (size)
            market_cap = info.get('marketCap', None)
            if market_cap:
                new_stock_data['size'] = market_cap
            else:
                new_stock_data['size'] = np.nan

            # Attempt to get beta
            beta = info.get('beta', np.nan)
            new_stock_data['beta'] = beta

            # 2. Financial Statements (yearly and quarterly)
            data_isy[f"{stock_id}_income_statement_yearly"] = ticker.income_stmt
            #data[f"{stock_id}_income_statement_quarterly"] = ticker.quarterly_income_stmt
            data_bsy[f"{stock_id}_balance_sheet_yearly"] = ticker.balance_sheet
            data_bsq[f"{stock_id}_balance_sheet_quarterly"] = ticker.quarterly_balance_sheet
            data_cy[f"{stock_id}_cashflow_yearly"] = ticker.cashflow
            #data[f"{stock_id}_cashflow_quarterly"] = ticker.quarterly_cashflow

            # 3. Analyst Data
            # data[f"{stock_id}_analyst_price_targets"] = ticker.analyst_price_targets
            data_ee[f"{stock_id}_earnings_estimates"] = ticker.earnings_estimate # 0007, 0009 don't have this
            data_re[f"{stock_id}_revenue_estimates"] = ticker.revenue_estimate
            #data[f"{stock_id}_earnings_history"] = ticker.earnings_history
            data_et[f"{stock_id}_eps_trend"] = ticker.eps_trend
            data_er[f"{stock_id}_eps_revisions"] = ticker.eps_revisions
            data_ge[f"{stock_id}_growth_estimates"] = ticker.growth_estimates
            data_r[f"{stock_id}_recommendations"] = ticker.recommendations

            # 4. Company Information
            data_i[f"{stock_id}_info"] = ticker.info
            data_s[f"{stock_id}_sustainability"] = ticker.sustainability
            data_n[f"{stock_id}_news"] = ticker.news

            # 5. Holders Information
            data_mh[f"{stock_id}_major_holders"] = ticker.major_holders
            data_ih[f"{stock_id}_institutional_holders"] = ticker.institutional_holders
            data_mh[f"{stock_id}_mutualfund_holders"] = ticker.mutualfund_holders
            data_ip[f"{stock_id}_insider_purchases"] = ticker.insider_purchases
            #data[f"{stock_id}_insider_transactions"] = ticker.insider_transactions
            #data[f"{stock_id}_insider_roster_holders"] = ticker.insider_roster_holders

            # 6. Other Data
            data_ed[f"{stock_id}_earnings_dates"] = ticker.earnings_dates
            data_c[f"{stock_id}_calendar"] = ticker.calendar
            #data[f"{stock_id}_sec_filings"] = ticker.sec_filings
            data_o[f"{stock_id}_options"] = ticker.options
            # data["shares"] = stock.shares
            data_sf[f"{stock_id}_shares_full"] = ticker.get_shares_full()
            data_fd[f"{stock_id}_funds_data"] = ticker.get_funds_data()

            all_data.append(new_stock_data)

        except Exception as e:
            print(f"Error processing {stock_id}: {e}")

    #if not all_data:
    #    return None, None, None,

    df = pd.concat(all_data)
    # df = df[['stock_id', 'Open', 'Close', 'Volume', 'industry', 'size', 'beta', 'returns']]

    return df, data_isy, data_bsy, data_bsq, data_cy, data_ee, data_re, data_et, data_er, data_ge, data_r, data_i, data_s, data_n, data_mh, data_ih, data_ip, data_ed, data_c, data_o, data_sf, data_fd, ticker

# Example usage
stock_ids = ['0005.HK', '0388.HK', '0939.HK', '1398.HK', '2318.HK', '2628.HK', '3968.HK', '3988.HK', '0175.HK', '0288.HK', '0386.HK', 
          '0700.HK', '0762.HK', '0857.HK', '0883.HK', '0941.HK', '0981.HK', '0992.HK', '1088.HK', '1093.HK', '1099.HK', '1211.HK', '1810.HK', '1876.HK', '1928.HK', '2015.HK', '2359.HK', '2899.HK', '3690.HK', '3692.HK', '6690.HK', '9618.HK', '9888.HK', '9961.HK', '9988.HK', '9999.HK']
start_date = '2020-03-24'
end_date = datetime.today().strftime('%Y-%m-%d') #current date.

price_data, data_isy, data_bsy, data_bsq, data_cy, data_ee, data_re, data_et, data_er, data_ge, data_r, data_i, data_s, data_n, data_mh, data_ih, data_ip, data_ed, data_c, data_o, data_sf, data_fd, stock = get_stock_data(stock_ids, start_date, end_date)

if price_data is not None:
    price_data = price_data.reset_index()
    print(price_data.head())

/Users/alanto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/alanto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['0005.HK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


No data found for 0005.HK


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['0388.HK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


No data found for 0388.HK


KeyboardInterrupt: 

# Final version

In [10]:
import ollama
import json
import datetime
import logging
import pandas as pd
import os
import re  # Import the re module for post-processing
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration
MODEL_NAME = "llama3.2"
ALPHA_FACTOR_DIR = "outputs"

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON). Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir="logs"):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

def load_alpha_factors(directory):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)

                if 'name' in df.columns and 'code' in df.columns:
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=MODEL_NAME):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input):
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            result = self.tools[tool_name](tool_input)
            self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
            return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

class MarketDebateAgent(Agent):
    """
    This agent orchestrates the debate between other agents.
    """

    def __init__(self):
        super().__init__("Market Debate Agent")
        self.analyst_agents = {  # Register analyst agents
            "fundamentals_analyst": FundamentalsAnalyst(),
            "sentiment_analyst": SentimentAnalyst(),
            "news_analyst": NewsAnalyst(),
            "technical_analyst": TechnicalAnalyst()
        }
        self.researcher_agents = { # Register researcher agents
            "bullish_researcher": BullishResearcher(),
            "bearish_researcher": BearishResearcher()
        }

    def conduct_debate(self, company_data, social_media_data, news_data, price_data, alpha_factors):
        """
        Orchestrates the market debate involving analyst and researcher agents.
        """
        debate_log = []

        # 1. Analyst Agents Provide Initial Analysis
        analyst_reports = {}
        for role, agent in self.analyst_agents.items():
            if role == "fundamentals_analyst":
                report = agent.analyze_fundamentals(company_data)
            elif role == "sentiment_analyst":
                report = agent.analyze_sentiment(social_media_data)
            elif role == "news_analyst":
                report = agent.analyze_news(news_data)
            elif role == "technical_analyst":
                report = agent.analyze_technical_indicators(price_data)
            else:
                continue  # Skip unknown roles
            analyst_reports[role] = report
            debate_log.append({"role": role, "report": report})
            self.log_action(f"{role} report", report)

        # 2. Researcher Agents Debate
        formatted_reports = f"""
        Fundamentals Report: {analyst_reports['fundamentals_analyst']}
        Sentiment Report: {analyst_reports['sentiment_analyst']}
        News Report: {analyst_reports['news_analyst']}
        Technical Report: {analyst_reports['technical_analyst']}
        """
        bullish_args = self.researcher_agents["bullish_researcher"].analyze_for_bullish_signals(
            formatted_reports)
        bearish_args = self.researcher_agents["bearish_researcher"].analyze_for_bearish_signals(
            formatted_reports)

        debate_log.append({"role": "bullish_researcher", "arguments": bullish_args})
        debate_log.append({"role": "bearish_researcher", "arguments": bearish_args})
        self.log_action("Bullish arguments", bullish_args)
        self.log_action("Bearish arguments", bearish_args)

        # 3. Alpha Factor Analysis (Integrated in Debate)
        alpha_analysis_prompt = f"""
        Given the analyst reports: 
        Fundamentals Report: {analyst_reports['fundamentals_analyst']}, 
        Sentiment Report: {analyst_reports['sentiment_analyst']}, 
        News Report: {analyst_reports['news_analyst']}, 
        Technical Report: {analyst_reports['technical_analyst']},
        the bullish arguments: {bullish_args}, 
        the bearish arguments: {bearish_args}, 
        and the following alpha factors: {json.dumps(alpha_factors)}, 
        which alpha factors should be selected or rejected and why?
        """
        alpha_analysis = self.generate_response(alpha_analysis_prompt)
        debate_log.append({"role": "debate_summary", "alpha_analysis": alpha_analysis})
        self.log_action("Alpha factor analysis", alpha_analysis)

        return debate_log

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")

    def analyze_fundamentals(self, company_data):
        """
        Analyze and evaluate company financials and stock performance. Then return a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        company_data should be a dict or DataFrame
        """
        prompt = f"""Analyze and evaluate company financials and stock performance, and provide a fundamentals analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Company Data: {company_data}"""
        report = self.generate_response(prompt)
        report = self.remove_code(report)  # Clean the output
        return report

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyze social media sentiment trends. Then return a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        social_media_data should be a string
        """
        prompt = f"""Analyze the following social media data/ social media sentiment trends, and provide a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Social Media Data: {social_media_data}"""
        sentiment = self.generate_response(prompt)
        sentiment = self.remove_code(sentiment)  # Clean the output
        return sentiment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyze global economic trends affecting markets. Then return a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        news_data should be a string
        """
        prompt = f"""Analyze the following news data to get the global economic trends affecting markets, and provide a report on potential market movements.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        News Data: {news_data}"""
        report = self.generate_response(prompt)
        report = self.remove_code(report)  # Clean the output
        return report

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes market trends from price data using technical indicators. Then return a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"""Analyze the following price data to get the market trend using technical indicators, and provide a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Price Data: {price_data}"""
        analysis = self.generate_response(prompt)
        analysis = self.remove_code(analysis)  # Clean the output
        return analysis

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"""Analyze the following reports to evaluate the investment potential of this company, and provide an assessment focused on bullish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment focused on bearish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"""Analyze the following reports to assess the risks of investing in this company, and provide an assessment focused on bearish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, debate_log):
        """
        Makes a trading decision based on the debate log.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        debate_log should be a list of dictionaries.
        """
        prompt = f"""Based on the market debate: {json.dumps(debate_log)}, 
        evaluate and make the trading decisions on market opportunities with a {self.risk_profile} risk profile.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        decision = self.generate_response(prompt)
        decision = self.remove_code(decision)  # Clean the output
        return decision

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision should be a string.
        """
        prompt = f"""Assess the risk associated with the following trading decision: {trading_decision}.
        Provide investment recommendations based on market analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        risk_assessment = self.generate_response(prompt)
        risk_assessment = self.remove_code(risk_assessment)  # Clean the output
        return risk_assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code
    
class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")
    
    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"""The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Approve or reject this trade.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        approval = self.generate_response(prompt)
        approval = self.remove_code(approval)  # Clean the output
        return approval

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

# --- Example Usage ---
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "market_debate_agent": MarketDebateAgent(),  # Debate orchestrator
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors(ALPHA_FACTOR_DIR)
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment"
    news_data = "Breaking: AAPL announces new product launch"
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    debate_log = agents["market_debate_agent"].conduct_debate(
        company_data, social_media_data, news_data, price_data, alpha_factors)  # Conduct the debate

    # Review alpha factor results
    for entry in debate_log:
        if entry["role"] == "debate_summary":
            alpha_analysis = entry["alpha_analysis"]
            print("Alpha Factor Analysis Results:")
            print(alpha_analysis)

    trading_decision = agents["trader_agent"].make_trading_decision(debate_log)  # Trader uses debate log
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("Trading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-21 13:50:59,587 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_5.csv'
2025-03-21 13:50:59,589 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_4.csv'
2025-03-21 13:50:59,592 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_1.csv'
2025-03-21 13:50:59,595 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_3.csv'
2025-03-21 13:50:59,597 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_2.csv'
2025-03-21 13:50:59,598 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-21 13:50:59,600 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-21 13:50:59,601 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-21 13:50:59,603 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'
2025-03-21 13:50:59,605 - INFO - Loaded alp

Alpha Factor Analysis Results:
Based on the provided code, it appears to be a technical analysis framework for financial markets. The code defines various alpha factors, each with its own formula and purpose.

To determine which alpha factors to select or reject, we need to consider several factors:

1. **Data quality and availability**: Some alpha factors rely on historical data, while others may require real-time data or proprietary inputs.
2. **Complexity and interpretability**: More complex formulas can be harder to understand and implement, making them less suitable for certain use cases.
3. **Market conditions and seasonality**: Alpha factors that perform well in one market condition or time period may not work as well in others.
4. **Risk management and position sizing**: Some alpha factors may require specific risk management strategies or position sizing techniques.

With these considerations in mind, here are some suggestions on which alpha factors to select or reject:

**Sel

2025-03-21 13:52:39,517 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-21 13:52:39,519 - INFO - Trader Agent: Generated response - Based on the market debate: [{"role": "fundamentals_analyst", "report": "**Fundamentals Analysis Report**\n\nBased on the provided company data for Apple Inc. (AAPL), this report aims to provide a qualitative analysis of the company's financial performance and stock market behavior.\n\n**Revenue Growth**\n\nThe revenue growth pattern over the quarters is as follows:\n\n- Q1 Revenue: $100 billion\n- Q2 Revenue: $105 billion\n\nThis indicates a year-over-year increase of 5% in Q2 revenue compared to Q1, which can be seen as a positive sign for the company's business continuity and profitability.\n\n**Debt Levels**\n\nThe debt level is given as $20 billion. This amount may seem relatively low for a multinational technology giant like Apple Inc., indicating that the company has a solid financial foundation. However, it's e

Trading Process Completed.


In [11]:
def review_alpha_factor_decision(debate_log):
    """
    Extracts and prints the alpha factor analysis from the debate log.
    """
    for entry in debate_log:
        if "role" in entry and entry["role"] == "debate_summary":
            if "alpha_analysis" in entry:
                return entry["alpha_analysis"]
            else:
                print("Alpha factor analysis key 'alpha_analysis' not found in 'debate_summary' entry.")
                return None  # Or consider raising an exception
    print("No 'debate_summary' entry found in debate log.")
    return None

alpha_factor_decision = review_alpha_factor_decision(debate_log)
if alpha_factor_decision:
    print("Final Alpha Factor Decision:")
    print(alpha_factor_decision)
else:
    print("Alpha factor decision not found in debate log.")

Final Alpha Factor Decision:
Based on the provided code, it appears to be a technical analysis framework for financial markets. The code defines various alpha factors, each with its own formula and purpose.

To determine which alpha factors to select or reject, we need to consider several factors:

1. **Data quality and availability**: Some alpha factors rely on historical data, while others may require real-time data or proprietary inputs.
2. **Complexity and interpretability**: More complex formulas can be harder to understand and implement, making them less suitable for certain use cases.
3. **Market conditions and seasonality**: Alpha factors that perform well in one market condition or time period may not work as well in others.
4. **Risk management and position sizing**: Some alpha factors may require specific risk management strategies or position sizing techniques.

With these considerations in mind, here are some suggestions on which alpha factors to select or reject:

**Selec

# Perfecting the multi-agents

In [6]:
import ollama
import json
import datetime
import logging
import pandas as pd
import os
import re  # Import the re module for post-processing
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration ---
CONFIG = {
    "MODEL_NAME": "llama3.2",
    "ALPHA_FACTOR_DIR": "outputs",
    "LOG_DIR": "logs"
}

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON). Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir=CONFIG["LOG_DIR"]):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

def load_alpha_factors(directory=CONFIG["ALPHA_FACTOR_DIR"]):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)

                if 'name' in df.columns and 'code' in df.columns:
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# --- Tool Definition ---
def get_current_date():
    """Returns the current date."""
    return datetime.date.today().isoformat()

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=CONFIG["MODEL_NAME"]):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input=None):  # Modified to have a default None for tool_input
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            tool_function = self.tools[tool_name]
            if tool_input is not None:
                result = tool_function(tool_input)
                self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
                return result
            else:
                result = tool_function()  # Call without arguments if no input provided
                self.log_action(f"Used tool: {tool_name}", f"Result: {result}")
                return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

class MarketDebateAgent(Agent):
    """
    This agent orchestrates the debate between other agents.
    """

    def __init__(self):
        super().__init__("Market Debate Agent")
        self.analyst_agents = {  # Register analyst agents
            "fundamentals_analyst": FundamentalsAnalyst(),
            "sentiment_analyst": SentimentAnalyst(),
            "news_analyst": NewsAnalyst(),
            "technical_analyst": TechnicalAnalyst()
        }
        self.researcher_agents = { # Register researcher agents
            "bullish_researcher": BullishResearcher(),
            "bearish_researcher": BearishResearcher()
        }

    def conduct_debate(self, company_data, social_media_data, news_data, price_data, alpha_factors, risk_aversion="moderate"):
        """
        Orchestrates the market debate involving analyst and researcher agents.
        """
        debate_log = []
        analyst_reports = {}

        # 1. Analyst Agents Provide Initial Analysis (with error handling and structured output for fundamentals)
        try:
            fundamentals_report = self.analyst_agents["fundamentals_analyst"].analyze_fundamentals(company_data)
            analyst_reports["fundamentals_analyst"] = fundamentals_report
            debate_log.append({"role": "fundamentals_analyst", "report": fundamentals_report})
            self.log_action("fundamentals_analyst report", fundamentals_report)
        except Exception as e:
            logging.error(f"Fundamentals analysis failed: {e}")
            analyst_reports["fundamentals_analyst"] = "Analysis failed."

        try:
            sentiment_report = self.analyst_agents["sentiment_analyst"].analyze_sentiment(social_media_data)
            analyst_reports["sentiment_analyst"] = sentiment_report
            debate_log.append({"role": "sentiment_analyst", "report": sentiment_report})
            self.log_action("sentiment_analyst report", sentiment_report)
        except Exception as e:
            logging.error(f"Sentiment analysis failed: {e}")
            analyst_reports["sentiment_analyst"] = "Analysis failed."

        try:
            news_report = self.analyst_agents["news_analyst"].analyze_news(news_data)
            analyst_reports["news_analyst"] = news_report
            debate_log.append({"role": "news_analyst", "report": news_report})
            self.log_action("news_analyst report", news_report)
        except Exception as e:
            logging.error(f"News analysis failed: {e}")
            analyst_reports["news_analyst"] = "Analysis failed."

        try:
            technical_report = self.analyst_agents["technical_analyst"].analyze_technical_indicators(price_data)
            analyst_reports["technical_analyst"] = technical_report
            debate_log.append({"role": "technical_analyst", "report": technical_report})
            self.log_action("technical_analyst report", technical_report)
        except Exception as e:
            logging.error(f"Technical analysis failed: {e}")
            analyst_reports["technical_analyst"] = "Analysis failed."

        # 2. Researcher Agents Debate (with basic dynamic interaction)
        formatted_reports = f"""
        Fundamentals Report: {analyst_reports.get('fundamentals_analyst', 'Not available')}
        Sentiment Report: {analyst_reports.get('sentiment_analyst', 'Not available')}
        News Report: {analyst_reports.get('news_analyst', 'Not available')}
        Technical Report: {analyst_reports.get('technical_analyst', 'Not available')}
        """

        # Example of basic dynamic interaction: Bullish researcher might focus on positive fundamentals
        bullish_prompt_suffix = ""
        if isinstance(analyst_reports.get('fundamentals_analyst'), dict) and analyst_reports['fundamentals_analyst'].get('overall_assessment') == 'positive':
            bullish_prompt_suffix = " Specifically focus on the positive aspects highlighted in the fundamentals report."

        bullish_args = self.researcher_agents["bullish_researcher"].analyze_for_bullish_signals(
            formatted_reports + bullish_prompt_suffix)
        bearish_args = self.researcher_agents["bearish_researcher"].analyze_for_bearish_signals(
            formatted_reports)

        debate_log.append({"role": "bullish_researcher", "arguments": bullish_args})
        debate_log.append({"role": "bearish_researcher", "arguments": bearish_args})
        self.log_action("Bullish arguments", bullish_args)
        self.log_action("Bearish arguments", bearish_args)

        # 3. Alpha Factor Analysis (Integrated in Debate)
        alpha_analysis_prompt = f"""
        Given the analyst reports:
        Fundamentals Report: {analyst_reports.get('fundamentals_analyst', 'Not available')},
        Sentiment Report: {analyst_reports.get('sentiment_analyst', 'Not available')},
        News Report: {analyst_reports.get('news_analyst', 'Not available')},
        Technical Report: {analyst_reports.get('technical_analyst', 'Not available')},
        the bullish arguments: {bullish_args},
        the bearish arguments: {bearish_args},
        and the following alpha factors: {json.dumps(alpha_factors)},

        Considering a risk aversion level of '{risk_aversion}',
        which alpha factors should be selected or rejected and why?
        For each potential alpha factor you select, please provide a confidence score (e.g., 0.0 to 1.0) indicating your certainty in its relevance.
        Strictly adhere to the following format for the potential and final alpha factors selected:

        Potential Alpha Factors:
        [
            {{
                "alpha name": "...",
                "alpha code": "...",
                "weight": ...,
                "confidence score": ...
            }},
            ...
        ]

        Final Alpha Factors Selected (those above a normal confidence threshold):
        [
            {{
                "alpha name": "...",
                "alpha code": "...",
                "weight": ...,
                "confidence score": ...
            }},
            ...
        ]

        Please ensure the 'confidence score' is a numerical value between 0.0 and 1.0.
        """
        alpha_analysis = self.generate_response(alpha_analysis_prompt)
        debate_log.append({"role": "debate_summary", "alpha_analysis": alpha_analysis})
        self.log_action("Alpha factor analysis", alpha_analysis)
        return debate_log

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")
        self.response_schemas = [
            ResponseSchema(name="overall_assessment", description="A brief overall assessment of the company's fundamentals (e.g., positive, negative, neutral)."),
            ResponseSchema(name="key_strengths", description="List of key fundamental strengths."),
            ResponseSchema(name="key_weaknesses", description="List of key fundamental weaknesses."),
            ResponseSchema(name="revenue_trend", description="Analysis of the company's revenue trend."),
            ResponseSchema(name="profitability", description="Analysis of the company's profitability."),
            ResponseSchema(name="debt_level", description="Assessment of the company's debt level."),
        ]
        self.output_parser = StructuredOutputParser.from_response_schemas(self.response_schemas)

    def analyze_fundamentals(self, company_data):
        """
        Analyze and evaluate company financials and stock performance. Then return a structured report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        company_data should be a dict or DataFrame
        """
        format_instructions = self.output_parser.get_format_instructions()
        prompt = f"""Analyze and evaluate the following company financials and stock performance, and provide a fundamentals analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Company Data: {company_data}
        {format_instructions}"""
        report = self.generate_response(prompt)
        try:
            parsed_report = self.output_parser.parse(report)
            return parsed_report
        except Exception as e:
            logging.error(f"Error parsing fundamentals report: {e}")
            return report

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyze social media sentiment trends. Then return a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        social_media_data should be a string
        """
        prompt = f"""Analyze the following social media data to determine the overall market sentiment and identify key positive and negative themes.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Social Media Data: {social_media_data}"""
        sentiment = self.generate_response(prompt)
        sentiment = self.remove_code(sentiment)  # Clean the output
        return sentiment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyze global economic trends affecting markets. Then return a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        news_data should be a string
        """
        prompt = f"""Analyze the following news data to identify global economic trends that could significantly affect financial markets. Report on potential market movements and key influencing factors.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        News Data: {news_data}"""
        report = self.generate_response(prompt)
        report = self.remove_code(report)  # Clean the output
        return report

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes market trends from price data using technical indicators. Then return a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"""Analyze the following price data to identify key market trends and potential buy/sell signals using common technical indicators (e.g., moving averages, RSI, MACD). Provide a technical analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Price Data: {price_data}"""
        analysis = self.generate_response(prompt)
        analysis = self.remove_code(analysis)  # Clean the output
        return analysis

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")
        self.add_tool("get_current_date", get_current_date)

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        current_date = self.use_tool("get_current_date") # Modified to call without argument
        prompt = f"""Analyze the following reports to evaluate the investment potential of this company, focusing on bullish signals and positive indicators. Consider the current market context as of {current_date}.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment focused on bearish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"""Analyze the following reports to assess the risks of investing in this company, focusing on bearish signals and negative indicators.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, debate_log, risk_preference="medium"):
        prompt = f"""Based on the market debate: {json.dumps(debate_log)},
        evaluate the overall sentiment and make a trading decision (buy, sell, or hold) considering a {risk_preference} risk profile.
        Justify your decision, including a confidence score (e.g., 1-10) for your recommendation.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        decision = self.generate_response(prompt)
        decision = self.remove_code(decision)  # Clean the output
        return decision

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision should be a string.
        """
        prompt = f"""Assess the risk associated with the following trading decision: {trading_decision}.
        Consider potential downsides and market volatility. Provide a risk assessment (e.g., low, medium, high) and any relevant recommendations.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        risk_assessment = self.generate_response(prompt)
        risk_assessment = self.remove_code(risk_assessment)  # Clean the output
        return risk_assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")

    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"""The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Based on this information, approve or reject this trade. Provide a brief justification.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        approval = self.generate_response(prompt)
        approval = self.remove_code(approval)  # Clean the output
        return approval

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

# --- Example Usage ---
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "market_debate_agent": MarketDebateAgent(),  # Debate orchestrator
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors()
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20, "profit_margin": 0.25}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment related to new product features and analyst upgrades."
    news_data = "Breaking: AAPL announces better-than-expected earnings and a new innovative product launch slated for next quarter. Global economic outlook remains cautiously optimistic."
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    debate_log = agents["market_debate_agent"].conduct_debate(
        company_data, social_media_data, news_data, price_data, alpha_factors, risk_aversion="high")  # Conduct the debate

    # Review alpha factor results
    for entry in debate_log:
        if entry["role"] == "debate_summary":
            alpha_analysis = entry["alpha_analysis"]
            print("Alpha Factor Analysis Results:")
            print(alpha_analysis)

    trading_decision = agents["trader_agent"].make_trading_decision(debate_log)  # Trader uses debate log
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("\nTrading Decision:", trading_decision)
    print("Risk Assessment:", risk_assessment)
    print("Fund Manager Approval:", approval)
    print("\nTrading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-26 16:20:45,992 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_5.csv'
2025-03-26 16:20:45,995 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_4.csv'
2025-03-26 16:20:45,998 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_1.csv'
2025-03-26 16:20:46,002 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_3.csv'
2025-03-26 16:20:46,005 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_2.csv'
2025-03-26 16:20:46,007 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-26 16:20:46,009 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-26 16:20:46,011 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-26 16:20:46,013 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'
2025-03-26 16:20:46,014 - INFO - Loaded alp

Alpha Factor Analysis Results:
Based on a risk aversion level of 'high', I've selected and rejected potential alpha factors from the given list. Here are my recommendations:

**Potential Alpha Factors:**

1. {
    "alpha name": "Bollinger Band Width Ratio",
    "alpha code": "\"((UPPER_BAND - LOWER_BAND) / SMA(CLOSE, 20)) * 100\"",
    "weight": 0.3,
    "confidence score": 0.8
}
2. {
    "alpha name": "Standard Deviation of Daily Returns",
    "alpha code": "STD(CLOSE, 1)",
    "weight": 0.4,
    "confidence score": 0.9
}
3. {
    "alpha name": "Average True Range (ATR) of High-Low Range",
    "alpha code": "\"(ATR(CLOSE, 14))\"",
    "weight": 0.5,
    "confidence score": 0.7
}
4. {
    "alpha name": "Force Index (FI)",
    "alpha code": "FI(CLOSE, VOL)",
    "weight": 0.2,
    "confidence score": 0.6
}

**Final Alpha Factors Selected:**

1. {
    "alpha name": "Standard Deviation of Daily Returns",
    "alpha code": "STD(CLOSE, 1)",
    "weight": 0.4,
    "confidence score": 0.9
}
2

2025-03-26 16:21:45,668 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 16:21:45,672 - INFO - Trader Agent: Generated response - Based on the market debate: [{"role": "fundamentals_analyst", "report": "```\n```json\n{\n  \"overall_assessment\": \"Positive\",\n  \"key_strengths\": \"Strong revenue growth, high profit margin, and low debt level indicate a financially healthy company.\",\n  \"key_weaknesses\": \"Limited information on the company's balance sheet or cash flow statement.\",\n  \"revenue_trend\": \"The company's revenue has increased by 5% from Q1 to Q2, indicating a stable growth trend.\",\n  \"profitability\": \"The high profit margin of 25% suggests that the company is efficiently converting its sales into profits.\",\n  \"debt_level\": \"The low debt level of $20 indicates that the company has minimal financial risk and can focus on long-term growth.\"\n}\n```"}, {"role": "sentiment_analyst", "report": "Based on the provided socia


Trading Decision: Based on the provided analysis, I evaluate the overall sentiment as neutral to slightly bullish, with some risk factors and opportunities present in the market.

Considering a medium risk profile, I recommend a **Buy** strategy with a confidence score of 7 out of 10. Here's my justification:

1. **Positive indicators:**
	* The moving average convergence divergence (MACD) is in a bullish trend, indicating potential upward momentum.
	* The RSI is slightly above the midpoint, suggesting that the market is entering a bullish phase.
2. **Risk factors:**
	* Ongoing trade tensions and Brexit uncertainty may impact investor confidence and global economic growth.
	* The RSI exceeding 70% could trigger a correction in the market.
3. **Opportunities:**
	* The Standard Deviation of Daily Returns and Average True Range (ATR) of High-Low Range alpha factors suggest that volatility is moderate to low, making it an attractive time for investors to buy into the market.
4. **Recommend

In [ ]:
# Review alpha factor results
for entry in debate_log:
    if entry["role"] == "debate_summary":
        alpha_analysis = entry["alpha_analysis"]
        print("\nRaw Alpha Factor Analysis Response:")
        print(alpha_analysis)  # Print the raw response for debugging

        try:
                parsed_analysis = json.loads(alpha_analysis)
                print("\nSuccessfully parsed alpha analysis as JSON.")
                if "final_alpha_factors_selected" in parsed_analysis:
                    final_alpha_factors = parsed_analysis["final_alpha_factors_selected"]
                    print("\nFinal Alpha Factors Selected (as list of dictionaries):")
                    print(json.dumps(final_alpha_factors, indent=4))

                    if final_alpha_factors:
                        try:
                            df_alpha_factors = pd.DataFrame(final_alpha_factors)
                            print("\nFinal Alpha Factors Selected (as Pandas DataFrame):")
                            print(df_alpha_factors)
                        except Exception as e_df:
                            print(f"\nCould not convert final alpha factors to DataFrame: {e_df}")
                            print(f"Content of final_alpha_factors: {final_alpha_factors}") # Inspect the content
                else:
                    print("\nKey 'final_alpha_factors_selected' not found in parsed alpha analysis.")

        except json.JSONDecodeError as e_json:
            print(f"\nError decoding alpha analysis JSON: {e_json}")
            print("\nAttempting to clean and parse the JSON response...")
            # Attempt to find the JSON object within the string using regex
            json_match = re.search(r"\{[\s\S]*\}", alpha_analysis)
            if json_match:
                cleaned_json_str = json_match.group(0)
                try:
                    parsed_analysis = json.loads(cleaned_json_str)
                    print("\nSuccessfully parsed cleaned JSON.")
                    if "final_alpha_factors_selected" in parsed_analysis:
                        final_alpha_factors = parsed_analysis["final_alpha_factors_selected"]
                        print("\nFinal Alpha Factors Selected (after cleaning):")
                        print(json.dumps(final_alpha_factors, indent=4))
                        try:
                            df_alpha_factors = pd.DataFrame(final_alpha_factors)
                            print("\nFinal Alpha Factors Selected (as Pandas DataFrame after cleaning):")
                            print(df_alpha_factors)
                        except Exception as e_df_cleaned:
                            print(f"\nCould not convert cleaned final alpha factors to DataFrame: {e_df_cleaned}")
                            print(f"Content of cleaned final_alpha_factors: {final_alpha_factors}") # Inspect cleaned content
                    else:
                        print("\nKey 'final_alpha_factors_selected' not found in cleaned parsed alpha analysis.")
                except json.JSONDecodeError as e_clean_json:
                    print(f"\nError decoding cleaned JSON: {e_clean_json}")
                    print(f"Cleaned JSON string: {cleaned_json_str}") # Inspect the cleaned string
            else:
                print("\nCould not find a JSON object within the response using regex.")


Raw Alpha Factor Analysis Response:
Based on a risk aversion level of 'high', I've selected and rejected potential alpha factors from the given list. Here are my recommendations:

**Potential Alpha Factors:**

1. {
    "alpha name": "Bollinger Band Width Ratio",
    "alpha code": "\"((UPPER_BAND - LOWER_BAND) / SMA(CLOSE, 20)) * 100\"",
    "weight": 0.3,
    "confidence score": 0.8
}
2. {
    "alpha name": "Standard Deviation of Daily Returns",
    "alpha code": "STD(CLOSE, 1)",
    "weight": 0.4,
    "confidence score": 0.9
}
3. {
    "alpha name": "Average True Range (ATR) of High-Low Range",
    "alpha code": "\"(ATR(CLOSE, 14))\"",
    "weight": 0.5,
    "confidence score": 0.7
}
4. {
    "alpha name": "Force Index (FI)",
    "alpha code": "FI(CLOSE, VOL)",
    "weight": 0.2,
    "confidence score": 0.6
}

**Final Alpha Factors Selected:**

1. {
    "alpha name": "Standard Deviation of Daily Returns",
    "alpha code": "STD(CLOSE, 1)",
    "weight": 0.4,
    "confidence score": 0

# Final one

In [41]:
import ollama
import json
import datetime
import logging
import pandas as pd
import os
import re  # Import the re module for post-processing
from langchain.output_parsers import StructuredOutputParser, ResponseSchema, OutputFixingParser
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama # Import Langchain's Ollama integration
from langchain_core.output_parsers import StrOutputParser # Import StrOutputParser

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuration ---
CONFIG = {
    "MODEL_NAME": "llama3.2",
    "ALPHA_FACTOR_DIR": "outputs",
    "LOG_DIR": "logs"
}

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON). Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir=CONFIG["LOG_DIR"]):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

def load_alpha_factors(directory=CONFIG["ALPHA_FACTOR_DIR"]):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)

                if 'name' in df.columns and 'code' in df.columns:
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# --- Tool Definition ---
def get_current_date():
    """Returns the current date."""
    return datetime.date.today().isoformat()

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=CONFIG["MODEL_NAME"]):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input=None):  # Modified to have a default None for tool_input
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            tool_function = self.tools[tool_name]
            if tool_input is not None:
                result = tool_function(tool_input)
                self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
                return result
            else:
                result = tool_function()  # Call without arguments if no input provided
                self.log_action(f"Used tool: {tool_name}", f"Result: {result}")
                return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class MarketDebateAgent(Agent):
    """
    This agent orchestrates the debate between other agents.
    """

    def __init__(self):
        super().__init__("Market Debate Agent")
        self.analyst_agents = {  # Register analyst agents
            "fundamentals_analyst": FundamentalsAnalyst(),
            "sentiment_analyst": SentimentAnalyst(),
            "news_analyst": NewsAnalyst(),
            "technical_analyst": TechnicalAnalyst()
        }
        self.researcher_agents = { # Register researcher agents
            "bullish_researcher": BullishResearcher(),
            "bearish_researcher": BearishResearcher()
        }
        self.alpha_factor_response_schemas = [
            ResponseSchema(name="alpha_name", description="The name of the alpha factor."),
            ResponseSchema(name="alpha_code", description="The code or formula of the alpha factor."),
            ResponseSchema(name="weight", description="The weight assigned to the alpha factor (numerical)."),
            ResponseSchema(name="confidence_score", description="The confidence score (0.0 to 1.0) for this alpha factor."),
        ]
        self.output_parser = StructuredOutputParser.from_response_schemas(self.alpha_factor_response_schemas)
        self.ollama_llm = ChatOllama(model=CONFIG["MODEL_NAME"]) # Initialize ChatOllama
        self.fixing_parser = OutputFixingParser.from_llm(self.ollama_llm, parser=self.output_parser, max_retries=3) # Use the ChatOllama instance

    def conduct_debate(self, company_data, social_media_data, news_data, price_data, alpha_factors, risk_aversion="moderate"):
        """
        Orchestrates the market debate involving analyst and researcher agents.
        """
        debate_log = []
        analyst_reports = {}
        parsed_output = None  # Initialize parsed_output

        # 1. Analyst Agents Provide Initial Analysis (with error handling and structured output for fundamentals)
        try:
            fundamentals_report = self.analyst_agents["fundamentals_analyst"].analyze_fundamentals(company_data)
            analyst_reports["fundamentals_analyst"] = fundamentals_report
            debate_log.append({"role": "fundamentals_analyst", "report": fundamentals_report})
            self.log_action("fundamentals_analyst report", fundamentals_report)
        except Exception as e:
            logging.error(f"Fundamentals analysis failed: {e}")
            analyst_reports["fundamentals_analyst"] = "Analysis failed."

        try:
            sentiment_report = self.analyst_agents["sentiment_analyst"].analyze_sentiment(social_media_data)
            analyst_reports["sentiment_analyst"] = sentiment_report
            debate_log.append({"role": "sentiment_analyst", "report": sentiment_report})
            self.log_action("sentiment_analyst report", sentiment_report)
        except Exception as e:
            logging.error(f"Sentiment analysis failed: {e}")
            analyst_reports["sentiment_analyst"] = "Analysis failed."

        try:
            news_report = self.analyst_agents["news_analyst"].analyze_news(news_data)
            analyst_reports["news_analyst"] = news_report
            debate_log.append({"role": "news_analyst", "report": news_report})
            self.log_action("news_analyst report", news_report)
        except Exception as e:
            logging.error(f"News analysis failed: {e}")
            analyst_reports["news_analyst"] = "Analysis failed."

        try:
            technical_report = self.analyst_agents["technical_analyst"].analyze_technical_indicators(price_data)
            analyst_reports["technical_analyst"] = technical_report
            debate_log.append({"role": "technical_analyst", "report": technical_report})
            self.log_action("technical_analyst report", technical_report)
        except Exception as e:
            logging.error(f"Technical analysis failed: {e}")
            analyst_reports["technical_analyst"] = "Analysis failed."

        # 2. Researcher Agents Debate (with basic dynamic interaction)
        formatted_reports = f"""
        Fundamentals Report: {analyst_reports.get('fundamentals_analyst', 'Not available')}
        Sentiment Report: {analyst_reports.get('sentiment_analyst', 'Not available')}
        News Report: {analyst_reports.get('news_analyst', 'Not available')}
        Technical Report: {analyst_reports.get('technical_analyst', 'Not available')}
        """

        # Example of basic dynamic interaction: Bullish researcher might focus on positive fundamentals
        bullish_prompt_suffix = ""
        if isinstance(analyst_reports.get('fundamentals_analyst'), dict) and analyst_reports['fundamentals_analyst'].get('overall_assessment') == 'positive':
            bullish_prompt_suffix = " Specifically focus on the positive aspects highlighted in the fundamentals report."

        bullish_args = self.researcher_agents["bullish_researcher"].analyze_for_bullish_signals(
            formatted_reports + bullish_prompt_suffix)
        bearish_args = self.researcher_agents["bearish_researcher"].analyze_for_bearish_signals(
            formatted_reports)

        debate_log.append({"role": "bullish_researcher", "arguments": bullish_args})
        debate_log.append({"role": "bearish_researcher", "arguments": bearish_args})
        self.log_action("Bullish arguments", bullish_args)
        self.log_action("Bearish arguments", bearish_args)

        # 3. Alpha Factor Analysis (Integrated in Debate)
        prompt_template = PromptTemplate(
            template="""Given the analyst reports:\n{analyst_reports}\nthe bullish arguments: {bullish_args}\nthe bearish arguments: {bearish_args}\nand the following alpha factors: {alpha_factors}\n\nConsidering a risk aversion level of '{risk_aversion}', and a normal confidence threshold of 0.6,\nwhich alpha factors should be selected or rejected and why?\nPlease output your response as a JSON object containing a list named 'final_alpha_factors_selected'. Each item in this list should be a JSON object conforming to the following schema:\n\n```json\n{{\n  "final_alpha_factors_selected": [\n    {{\n      "alpha_name": "...",\n      "alpha_code": "...",\n      "weight": ...,\n      "confidence_score": ...\n    }},\n    ...\n  ]\n}}\n```\n""",
            input_variables=["analyst_reports", "bullish_args", "bearish_args", "alpha_factors", "risk_aversion"]
        )

        prompt = prompt_template.format(
            analyst_reports=formatted_reports,
            bullish_args=bullish_args,
            bearish_args=bearish_args,
            alpha_factors=json.dumps(alpha_factors),
            risk_aversion=risk_aversion
        )

        alpha_analysis_str = self.generate_response(prompt)
        parsed_output = None # Initialize parsed_output before the try block
        try:
            # Attempt to parse the response directly as a JSON object
            alpha_analysis_json = json.loads(alpha_analysis_str)
            if "final_alpha_factors_selected" in alpha_analysis_json:
                parsed_output = {"final_alpha_factors_selected": alpha_analysis_json["final_alpha_factors_selected"]}
                debate_log.append({"role": "debate_summary", "alpha_analysis": parsed_output})
                self.log_action("Alpha factor analysis (parsed)", parsed_output)
            else:
                raise ValueError("Key 'final_alpha_factors_selected' not found in JSON response.")
        except json.JSONDecodeError:
            logging.error(f"Error decoding JSON for alpha analysis. Attempting to fix.")
            try:
                fixed_output = self.fixing_parser.parse(alpha_analysis_str)
                debate_log.append({"role": "debate_summary", "alpha_analysis": fixed_output})
                self.log_action("Alpha factor analysis (fixed)", fixed_output)
                parsed_output = fixed_output # Assign fixed output
            except Exception as fix_e:
                logging.error(f"Error fixing alpha analysis: {fix_e}")
                debate_log.append({"role": "debate_summary", "alpha_analysis": alpha_analysis_str})
                self.log_action("Alpha factor analysis (raw)", alpha_analysis_str)
                parsed_output = None # Assign a default value even if fixing fails
        except ValueError as ve:
            logging.error(f"Error parsing alpha analysis: {ve}")
            try:
                fixed_output = self.fixing_parser.parse(alpha_analysis_str)
                debate_log.append({"role": "debate_summary", "alpha_analysis": fixed_output})
                self.log_action("Alpha factor analysis (fixed)", fixed_output)
                parsed_output = fixed_output # Assign fixed output
            except Exception as fix_e:
                logging.error(f"Error fixing alpha analysis: {fix_e}")
                debate_log.append({"role": "debate_summary", "alpha_analysis": alpha_analysis_str})
                self.log_action("Alpha factor analysis (raw)", alpha_analysis_str)
                parsed_output = None # Assign a default value even if fixing fails

        return debate_log, parsed_output

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")
        self.response_schemas = [
            ResponseSchema(name="overall_assessment", description="A brief overall assessment of the company's fundamentals (e.g., positive, negative, neutral)."),
            ResponseSchema(name="key_strengths", description="List of key fundamental strengths."),
            ResponseSchema(name="key_weaknesses", description="List of key fundamental weaknesses."),
            ResponseSchema(name="revenue_trend", description="Analysis of the company's revenue trend."),
            ResponseSchema(name="profitability", description="Analysis of the company's profitability."),
            ResponseSchema(name="debt_level", description="Assessment of the company's debt level."),
        ]
        self.output_parser = StructuredOutputParser.from_response_schemas(self.response_schemas)

    def analyze_fundamentals(self, company_data):
        """
        Analyze and evaluate company financials and stock performance. Then return a structured report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        company_data should be a dict or DataFrame
        """
        format_instructions = self.output_parser.get_format_instructions()
        prompt = f"""Analyze and evaluate the following company financials and stock performance, and provide a fundamentals analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Company Data: {company_data}
        {format_instructions}"""
        report = self.generate_response(prompt)
        try:
            parsed_report = self.output_parser.parse(report)
            return parsed_report
        except Exception as e:
            logging.error(f"Error parsing fundamentals report: {e}")
            return report

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyze social media sentiment trends. Then return a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        social_media_data should be a string
        """
        prompt = f"""Analyze the following social media data to determine the overall market sentiment and identify key positive and negative themes.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Social Media Data: {social_media_data}"""
        sentiment = self.generate_response(prompt)
        sentiment = self.remove_code(sentiment)  # Clean the output
        return sentiment

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyze global economic trends affecting markets. Then return a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        news_data should be a string
        """
        prompt = f"""Analyze the following news data to identify global economic trends that could significantly affect financial markets. Report on potential market movements and key influencing factors.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        News Data: {news_data}"""
        report = self.generate_response(prompt)
        report = self.remove_code(report)  # Clean the output
        return report

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes market trends from price data using technical indicators. Then return a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"""Analyze the following price data to identify key market trends and potential buy/sell signals using common technical indicators (e.g., moving averages, RSI, MACD). Provide a technical analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Price Data: {price_data}"""
        analysis = self.generate_response(prompt)
        analysis = self.remove_code(analysis)  # Clean the output
        return analysis

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")
        self.add_tool("get_current_date", get_current_date)

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        current_date = self.use_tool("get_current_date") # Modified to call without argument
        prompt = f"""Analyze the following reports to evaluate the investment potential of this company, focusing on bullish signals and positive indicators. Consider the current market context as of {current_date}.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment focused on bearish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"""Analyze the following reports to assess the risks of investing in this company, focusing on bearish signals and negative indicators.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, debate_log):
        """
        Makes a trading decision based on the debate log.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        debate_log should be a list of dictionaries.
        """
        prompt = f"""Based on the market debate: {json.dumps(debate_log)},
        evaluate the overall sentiment and make a trading decision (buy, sell, or hold) considering a {self.risk_profile} risk profile. Justify your decision.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        decision = self.generate_response(prompt)
        decision = self.remove_code(decision)  # Clean the output
        return decision

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision should be a string.
        """
        prompt = f"""Assess the risk associated with the following trading decision: {trading_decision}.
        Consider potential downsides and market volatility. Provide a risk assessment (e.g., low, medium, high) and any relevant recommendations.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        risk_assessment = self.generate_response(prompt)
        risk_assessment = self.remove_code(risk_assessment)  # Clean the output
        return risk_assessment

class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")

    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"""The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Based on this information, approve or reject this trade. Provide a brief justification.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        approval = self.generate_response(prompt)
        approval = self.remove_code(approval)  # Clean the output
        return approval

# --- Example Usage ---
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "market_debate_agent": MarketDebateAgent(),  # Debate orchestrator
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors()
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20, "profit_margin": 0.25}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment related to new product features and analyst upgrades."
    news_data = "Breaking: AAPL announces better-than-expected earnings and a new innovative product launch slated for next quarter. Global economic outlook remains cautiously optimistic."
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    debate_log, parsed_alpha_output = agents["market_debate_agent"].conduct_debate(
        company_data, social_media_data, news_data, price_data, alpha_factors, risk_aversion="high")  # Conduct the debate

    final_alpha_factors_list = []
    for entry in debate_log:
        if entry["role"] == "debate_summary":
            alpha_analysis = entry["alpha_analysis"]
            print("\nRaw Alpha Factor Analysis Response:")
            print(alpha_analysis)

            if isinstance(alpha_analysis, dict):
                if "final_alpha_factors_selected" in alpha_analysis:
                    final_alpha_factors_list = alpha_analysis["final_alpha_factors_selected"]
                    print("\nFinal Alpha Factors Selected (as list of dictionaries):")
                    print(json.dumps(final_alpha_factors_list, indent=4))
                    break
                elif "alpha_name" in alpha_analysis and "alpha_code" in alpha_analysis and "weight" in alpha_analysis and "confidence_score" in alpha_analysis:
                    # Handle the case where the output is a single alpha factor dictionary
                    final_alpha_factors_list = [alpha_analysis]
                    print("\nFinal Alpha Factors Selected (as list of dictionaries - single factor detected):")
                    print(json.dumps(final_alpha_factors_list, indent=4))
                    break
                else:
                    print("\nAlpha analysis dictionary did not contain the expected keys ('final_alpha_factors_selected' or individual alpha factor keys).")
            else:
                print("\nAlpha analysis was not in the expected dictionary format.")
            break

    if final_alpha_factors_list:
        try:
            df_alpha_factors = pd.DataFrame(final_alpha_factors_list)
            print("\nFinal Alpha Factors Selected (as Pandas DataFrame):")
            print(df_alpha_factors)
        except Exception as e:
            print(f"\nCould not convert final alpha factors to DataFrame: {e}")

    trading_decision = agents["trader_agent"].make_trading_decision(debate_log)  # Trader uses debate log
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("\nTrading Decision:", trading_decision)
    print("Risk Assessment:", risk_assessment)
    print("Fund Manager Approval:", approval)
    print("\nTrading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-26 18:34:51,409 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_5.csv'
2025-03-26 18:34:51,412 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_4.csv'
2025-03-26 18:34:51,414 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_1.csv'
2025-03-26 18:34:51,417 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_3.csv'
2025-03-26 18:34:51,420 - INFO - Loaded alpha factors from 'outputs/HK_specific_per_domain_result_2.csv'
2025-03-26 18:34:51,422 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-26 18:34:51,423 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-26 18:34:51,425 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-26 18:34:51,426 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'
2025-03-26 18:34:51,428 - INFO - Loaded alp


Raw Alpha Factor Analysis Response:
{'alpha_name': 'Gross Margin Growth Rate', 'alpha_code': 'GMGR', 'weight': '0.4', 'confidence_score': '0.62'}

Final Alpha Factors Selected (as list of dictionaries - single factor detected):
[
    {
        "alpha_name": "Gross Margin Growth Rate",
        "alpha_code": "GMGR",
        "weight": "0.4",
        "confidence_score": "0.62"
    }
]

Final Alpha Factors Selected (as Pandas DataFrame):
                 alpha_name alpha_code weight confidence_score
0  Gross Margin Growth Rate       GMGR    0.4             0.62


2025-03-26 18:35:57,844 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:35:57,847 - INFO - Trader Agent: Generated response - Based on the market debate: [{"role": "fundamentals_analyst", "report": {"overall_assessment": "Positive", "key_strengths": "Consistent quarterly revenue growth, healthy profit margin.", "key_weaknesses": "Moderate debt levels, no information on cash reserves.", "revenue_trend": "Growing revenue with slight increases in each quarter, indicating a stable and expanding business.", "profitability": "Profit margins are well-maintained at 25%, suggesting efficient operations and strong demand for the company's products.", "debt_level": "Debt levels of $20 are moderate and do not pose an immediate risk to the company's financial stability, but it would be beneficial to see improved cash reserves."}}, {"role": "sentiment_analyst", "report": "Based on the social media data, here's a qualitative analysis of the overall market s


Trading Decision: Based on the analysis reports provided, I will attempt to summarize the key points and provide a trading decision.

**Positive Indicators:**

* The company has strong fundamentals, including consistent quarterly revenue growth and healthy profit margins.
* The Sentiment Report shows an overwhelmingly positive market sentiment towards the company, with investors and enthusiasts expressing excitement around new product features, analyst upgrades, and investor enthusiasm.
* The News Report highlights a cautiously optimistic global economic outlook, which could support equity markets, particularly for technology stocks like Apple's (AAPL).
* The Technical Report suggests a bullish short-term trend, with prices steadily increasing over the past few days.

**Negative Indicators:**

* Moderate debt levels of $20, which could pose a risk to the company's financial stability if not managed properly.
* No information on cash reserves, which could indicate potential liquidity i

In [42]:
df_alpha_factors

,alpha_name,alpha_code,weight,confidence_score
0,Gross Margin Growth Rate,GMGR,0.4,0.62
